# Init

In [21]:
# imports
import gurobipy as gp
from gurobipy import GRB
from collections import defaultdict
import numpy as np

In [22]:
OPTIMAL_SOLUTION = 0 # used to evaluate gap between math model and heuristics

---
#### Load data

In [23]:
num_video = 0
num_endpoint = 0
num_req_descriptions = 0
num_server = 0

cache_capacity = 0
video_size = []

latency = defaultdict(lambda: defaultdict(int))     # [endpoint][cache/datacenter] = latency
reqs = defaultdict(lambda: defaultdict(int))        # [endpoint][video] = num reqs

# dataset = "dataset/videos_worth_spreading.in"
# dataset = "dataset/me_at_the_zoo.in"
dataset = "dataset/custom.in"
# dataset = "dataset/minimal.in"


In [24]:
status = 0
endpoint_index = 0
num_connected_cache = 0
with open(dataset, "r") as f:
    for line_content in f:
        line = line_content.split()

        if status ==0:                                  # get counters
            num_video = int(line[0])
            num_endpoint = int(line[1])
            num_req_descriptions = int(line[2])
            num_server = int(line[3])
            cache_capacity = int(line[4])
            status = 1

        elif status == 1:                               # get video dims
            for size in line:
                video_size.append(int(size))
            status = 2

        elif status == 2:                               # get datacenter latency and connected cache number
            data_center_latency = int(line[0])
            latency[endpoint_index][num_server] = data_center_latency
            
            num_connected_cache = int(line[1])
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
            else:
                status = 3
        
        elif status == 3:                                  # get cache latency
            cache_index = int(line[0])
            cache_latency = int(line[1])
            latency[endpoint_index][cache_index] = cache_latency
            
            num_connected_cache = num_connected_cache - 1
            if not num_connected_cache:
                endpoint_index = endpoint_index + 1
                if endpoint_index == num_endpoint:
                    status = 4
                else:
                    status = 2
        
        elif status == 4:                                   # take num requests
            video_index = int(line[0])
            endpoint_index = int(line[1])
            num_reqs = int(line[2])
            reqs[endpoint_index][video_index] = num_reqs                         

In [25]:
print(f"num video: {num_video}, num endpoints {num_endpoint}, req descriptions {num_req_descriptions}, num cache {num_server}, dim {cache_capacity}")
print(f"video sized: {video_size}")
print(f"latencies: {latency}")
print(f"reqs: {reqs}")

num video: 5, num endpoints 2, req descriptions 4, num cache 3, dim 100
video sized: [50, 50, 80, 30, 110]
latencies: defaultdict(<function <lambda> at 0x7f34ec5adb20>, {0: defaultdict(<class 'int'>, {3: 1000, 0: 100, 2: 200, 1: 300}), 1: defaultdict(<class 'int'>, {3: 500})})
reqs: defaultdict(<function <lambda> at 0x7f34d511e660>, {0: defaultdict(<class 'int'>, {3: 1500, 4: 500, 1: 1000}), 1: defaultdict(<class 'int'>, {0: 1000})})


---
# Math model for Guroby

In [26]:
endpoint_index = range(num_endpoint)
server_index = range(num_server + 1) # I've modelled datacenter as last server
video_index = range(num_video)

In [27]:
model = gp.Model("YoutubeCache")

# decision vars
x = model.addVars(endpoint_index, server_index, video_index, vtype=gp.GRB.BINARY, name="x")
y = model.addVars(server_index, video_index, vtype=gp.GRB.BINARY, name="y")

# objective function
obj = gp.quicksum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
# the + y[s,v] it's used just to not let place useless video in cache server (but is not needed for this problem)
# obj = gp.quicksum((latency[e][s]*reqs[e][v]*x[e,s,v] + y[s,v])for e in endpoint_index for s in server_index for v in video_index)
model.setObjective(obj, GRB.MINIMIZE)
# model.setObjective(gp.quicksum(y[s,v] for s in server_index for v in video_index), GRB.MAXIMIZE)


# constraints
constr = (gp.quicksum(x[e,s,v] for e in endpoint_index)  <= num_endpoint*y[s,v] for s in server_index for v in video_index )
model.addConstrs(constr, name="if video v available on server s")

constr = ( gp.quicksum( x[e,s,v] for s in server_index ) == (1 if reqs[e][v] else 0) for e in endpoint_index for v in video_index ) # datacenter excluded 
# constr = ( gp.quicksum( x[e,s,v] for s in latency[e] ) == (1 if reqs[e][v] else 0) for e in endpoint_index for v in video_index ) # latency[e] directly check existring connections
model.addConstrs(constr, name="every request must be satisfied")

constr = ( gp.quicksum(video_size[v] * y[s,v] for v in video_index) <= cache_capacity for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="cache capacity")


constr = ( y[num_server,v] == 1 for v in video_index ) # cache servers are from 0 to s-1, s index (num_server) is for datacenter
model.addConstrs(constr, name="Datacenter have all videos")

constr = ( gp.quicksum( x[e,s,v] for v in video_index ) <= (num_video*latency[e][s]) for e in endpoint_index for s in server_index[:-1] ) # -1 because datacenter have all the video
model.addConstrs(constr, name="video v must be available on server s to be selected")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>}

In [28]:
# Optimize the model
model.optimize()

# model.display()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 44 rows, 60 columns and 150 nonzeros
Model fingerprint: 0x198f7325
Variable types: 0 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+04, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 2600000.0000
Presolve removed 44 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 1.25e+06 2.6e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.250000000000e+06, best bound 1.250000000000e+06, gap 0.0000%


In [29]:
def print_full_output():
    print("Optimal X [endpoint, server, video] values:")
    for e in endpoint_index:
        for s in server_index:
            for v in video_index:
                print(f"X[{e},{s},{v}] * {latency[e][s]} = {x[e,s,v]}")
    print("\nOptimal Y [server, video] values:")
    for s in server_index:
        for v in video_index:
            print(f"Y[{s},{v}] = {y[s,v]}")

def print_concise_output():
    print("Optimal X [endpoint, server, video] values:")
    for e in endpoint_index:
        for s in server_index:
            for v in video_index:
                if x[e,s,v].x:
                    print(f"X[{e},{s},{v}] * {latency[e][s]} = {x[e,s,v]}")
    print("\nOptimal Y [server, video] values:")
    for s in server_index:
        for v in video_index:
            if y[s,v].x:
                print(f"Y[{s},{v}] = {y[s,v]}")

In [30]:
# results
if model.status == gp.GRB.OPTIMAL:
    print("\nOptimization successful!")
    # print_full_output()
    print_concise_output()
    print(f"\nOptimal objective value: {model.objVal}")
    OPTIMAL_SOLUTION = model.ObjVal
elif model.status == gp.GRB.INFEASIBLE:
    print("Model is infeasible.")
elif model.status == gp.GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization ended with status {model.status}")


Optimization successful!
Optimal X [endpoint, server, video] values:
X[0,0,1] * 100 = <gurobi.Var x[0,0,1] (value 1.0)>
X[0,0,3] * 100 = <gurobi.Var x[0,0,3] (value 1.0)>
X[0,3,4] * 1000 = <gurobi.Var x[0,3,4] (value 1.0)>
X[1,3,0] * 500 = <gurobi.Var x[1,3,0] (value 1.0)>

Optimal Y [server, video] values:
Y[0,1] = <gurobi.Var y[0,1] (value 1.0)>
Y[0,3] = <gurobi.Var y[0,3] (value 1.0)>
Y[1,1] = <gurobi.Var y[1,1] (value 1.0)>
Y[1,3] = <gurobi.Var y[1,3] (value 1.0)>
Y[2,1] = <gurobi.Var y[2,1] (value 1.0)>
Y[2,3] = <gurobi.Var y[2,3] (value 1.0)>
Y[3,0] = <gurobi.Var y[3,0] (value 1.0)>
Y[3,1] = <gurobi.Var y[3,1] (value 1.0)>
Y[3,2] = <gurobi.Var y[3,2] (value 1.0)>
Y[3,3] = <gurobi.Var y[3,3] (value 1.0)>
Y[3,4] = <gurobi.Var y[3,4] (value 1.0)>

Optimal objective value: 1250000.0


---
# heuristics

## Basics

### place video with highest request number in nearest cache when possible and place all videos for first endpoint

In [31]:
# 1. sort by request number. use a list containing sorted indexes
sorted_reqs = []
E_IND = 0
V_IND = 1

for e in range(len(reqs)):
    # Sort v indices by descending value for this e
    sorted_vs = sorted(
        [v for v in reqs[e] if reqs[e][v] != 0],
        # range(len(reqs[e])),
        key=lambda v: reqs[e][v],
        reverse=True
    )
    # Add (e, v) pairs to the list
    sorted_reqs.extend((e, v) for v in sorted_vs)

# List to store sorted (e, v) pairs
sorted_latency = defaultdict(list)

for e in latency:
    # Sort v indices by descending value for this e
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    # Add (e, v) pairs to the list
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity
# print(curr_capacity)

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos

for req in sorted_reqs:
    # print(req)
    req_endpoint = req[E_IND]
    req_video = req[V_IND]
    req_video_size = video_size[req_video]

    for curr_cache_index in sorted_latency[req_endpoint]:
        if y[curr_cache_index, req_video]:
            x[req_endpoint, curr_cache_index, req_video] = 1
            break
        else:
            if curr_capacity[curr_cache_index] > req_video_size:
                curr_capacity[curr_cache_index] -= req_video_size
                y[curr_cache_index, req_video] = 1
                x[req_endpoint, curr_cache_index, req_video] = 1
                break



# print(reqs)
# print(sorted_indexes)
# print(sorted_latency)
# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = sum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")


APPROX RESULT: 1250000.0 - GAP: 0.0% - OPTIMAL RESULT 1250000.0


### ROUND ROBIN: place video with highest request number in nearest cache (every iteration change endpoint)

In [32]:
# 1. sort by request number. use a list containing sorted indexes
sorted_reqs = defaultdict(list)
E_IND = 0
V_IND = 1

for e in range(len(reqs)):
    sorted_vs = sorted(
        [v for v in reqs[e] if reqs[e][v] != 0],
        key=lambda v: reqs[e][v],
        reverse=True
    )
    sorted_reqs[e] = sorted_vs

# List to store sorted (e, v) pairs
sorted_latency = defaultdict(list)

for e in latency:
    # Sort v indices by descending value for this e
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    # Add (e, v) pairs to the list
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity
# print(curr_capacity)

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos

Done = False
endpoints_req_index = [0 for _ in server_index]
while not Done:
    Done = True
    for curr_endpoint in endpoint_index:
        curr_endpoint_req_index = endpoints_req_index[curr_endpoint]
        
        if curr_endpoint_req_index < len(sorted_reqs[curr_endpoint]):
            Done = False # There could still be reqs not satisfied other than this
            req_video = sorted_reqs[curr_endpoint][curr_endpoint_req_index]
            req_video_size = video_size[req_video]
            # print(req_video)

            for curr_cache_index in sorted_latency[curr_endpoint]:
                if y[curr_cache_index, req_video]:
                    x[curr_endpoint, curr_cache_index, req_video] = 1
                    break
                else:
                    if curr_capacity[curr_cache_index] > req_video_size:
                        curr_capacity[curr_cache_index] -= req_video_size
                        y[curr_cache_index, req_video] = 1
                        x[curr_endpoint, curr_cache_index, req_video] = 1
                        break
                    
        endpoints_req_index[curr_endpoint] += 1

# print(reqs)
# print(sorted_indexes)
# print(sorted_latency)
# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = sum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")


APPROX RESULT: 1250000.0 - GAP: 0.0% - OPTIMAL RESULT 1250000.0


### place video with highest request number in nearest cache when possible (order over both endpoint and video, practically order by absolute req. number)

In [33]:
# 1. sort by request number. use a list containing sorted indexes
sorted_reqs = []
E_IND = 0
V_IND = 1

sorted_reqs = sorted(
    [(e, v) for e in range(len(reqs)) for v in reqs[e] if reqs[e][v] != 0],
    key=lambda pair: reqs[pair[0]][pair[1]],
    reverse=True
)
# print(sorted_reqs)

# List to store sorted (e, v) pairs
sorted_latency = defaultdict(list)

for e in latency:
    # Sort v indices by descending value for this e
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    # Add (e, v) pairs to the list
    sorted_latency[e] = sorted_s


# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity
# print(curr_capacity)

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos

for req_endpoint,req_video in sorted_reqs:
    # print(req)
    req_video_size = video_size[req_video]

    for curr_cache_index in sorted_latency[req_endpoint]:
        if y[curr_cache_index, req_video]:
            x[req_endpoint, curr_cache_index, req_video] = 1
            break
        else:
            if curr_capacity[curr_cache_index] > req_video_size:
                curr_capacity[curr_cache_index] -= req_video_size
                y[curr_cache_index, req_video] = 1
                x[req_endpoint, curr_cache_index, req_video] = 1
                break


# print(reqs)
# print(sorted_indexes)
# print(sorted_latency)
# print("X")
# print(x)
# print("Y")
# print(y)
APPROX_RESULT = sum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")


APPROX RESULT: 1250000.0 - GAP: 0.0% - OPTIMAL RESULT 1250000.0


### Place video ordered by popularity in best cache

In [ ]:
# 1. sort by request number. use a list containing sorted indexes
video_req_count = [0 for _ in video_index]
latency_sum = defaultdict(lambda: defaultdict(lambda: defaultdict(int))) # used to find best cache to place a video [server][video][latency sum / num endpoint / score]
LATENCY_INDEX = 0
NUM_ENDPOINT_CONNECTED_INDEX = 1
SCORE_INDEX = 2

print(reqs)
for curr_endpoint_index, endpoint_reqs in reqs.items():
    # print(f"endpoint_index: {curr_endpoint_index}")
    for curr_video_index, req_num in endpoint_reqs.items():
        # print(f"  Video: {curr_video_index}, Num reqs: {req_num}")
        video_req_count[curr_video_index] += req_num
        for curr_server_index, lat in latency[curr_endpoint_index].items():
            if lat:
                latency_sum[curr_server_index][curr_video_index][LATENCY_INDEX] += lat
                latency_sum[curr_server_index][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX] += 1
                latency_sum[curr_server_index][curr_video_index][SCORE_INDEX] = (
                    latency_sum[curr_server_index][curr_video_index][NUM_ENDPOINT_CONNECTED_INDEX] 
                    /
                    latency_sum[curr_server_index][curr_video_index][LATENCY_INDEX]
                )
            
sorted_video_indexes = sorted(range(num_video), key=lambda i: video_req_count[i], reverse=True)
print(latency_sum)

# use a list to keep current cache capacity (will be decreased every time a video is placed in cache)
curr_capacity = [cache_capacity for _ in range(num_server)]
curr_capacity.append(float('inf')) # datacenter doesn't have capacity
# print(curr_capacity)

# create vars (simil guroby, used numpy for efficiency)
x = np.zeros((num_endpoint, (num_server+1), num_video)) # e take v from s
y = np.zeros(((num_server+1), num_video)) # v is in s
y[num_server, :] = 1 # datacenter keep all the videos

sorted_latency = defaultdict(list)

for e in latency:
    # Sort v indices by ascending value for this e
    sorted_s = sorted(
        [s for s in latency[e] if latency[e][s] != 0],
        key=lambda s: latency[e][s]
    )
    # Add (e, v) pairs to the list
    sorted_latency[e] = sorted_s

#TODO: check why video 0 is placed on cache 0 that is useless

for curr_video_index in sorted_video_indexes:
    for curr_server_index in sorted(
        range(len(latency_sum)),
        key=lambda i: latency_sum[i][curr_video_index][SCORE_INDEX],
        reverse=True
    ):
        if not y[curr_server_index, curr_video_index]:
            if curr_capacity[curr_server_index] > video_size[curr_video_index]:
                curr_capacity[curr_server_index] -= video_size[curr_video_index]
                y[curr_server_index, curr_video_index] = 1
                break

for req_endpoint,req_videos in reqs.items():    
    for curr_video_index in req_videos:
        if reqs[req_endpoint][curr_video_index]:
            for curr_server_index in sorted_latency[req_endpoint]:
                if y[curr_server_index, curr_video_index]:
                    x[req_endpoint, curr_server_index, curr_video_index] = 1
                    break

print("X")
print(x)
print("Y")
print(y)
APPROX_RESULT = sum(latency[e][s]*reqs[e][v]*x[e,s,v] for e in endpoint_index for s in server_index for v in video_index)
GAP = ( abs(OPTIMAL_SOLUTION - APPROX_RESULT) / OPTIMAL_SOLUTION ) * 100
print(f"APPROX RESULT: {APPROX_RESULT} - GAP: {GAP}% - OPTIMAL RESULT {OPTIMAL_SOLUTION}")


defaultdict(<function <lambda> at 0x7f34d511e660>, {0: defaultdict(<class 'int'>, {3: 1500, 4: 500, 1: 1000, 0: 0, 2: 0}), 1: defaultdict(<class 'int'>, {0: 1000, 1: 0, 2: 0, 3: 0, 4: 0})})
defaultdict(<function <lambda> at 0x7f34c1b86de0>, {3: defaultdict(<function <lambda>.<locals>.<lambda> at 0x7f34c1b8e3e0>, {3: defaultdict(<class 'int'>, {0: 1500, 1: 2, 2: 0.0013333333333333333}), 4: defaultdict(<class 'int'>, {0: 1500, 1: 2, 2: 0.0013333333333333333}), 1: defaultdict(<class 'int'>, {0: 1500, 1: 2, 2: 0.0013333333333333333}), 0: defaultdict(<class 'int'>, {0: 1500, 1: 2, 2: 0.0013333333333333333}), 2: defaultdict(<class 'int'>, {0: 1500, 1: 2, 2: 0.0013333333333333333})}), 0: defaultdict(<function <lambda>.<locals>.<lambda> at 0x7f34c1b8eca0>, {3: defaultdict(<class 'int'>, {0: 100, 1: 1, 2: 0.01}), 4: defaultdict(<class 'int'>, {0: 100, 1: 1, 2: 0.01}), 1: defaultdict(<class 'int'>, {0: 100, 1: 1, 2: 0.01}), 0: defaultdict(<class 'int'>, {0: 100, 1: 1, 2: 0.01}), 2: defaultdict(<

### place video based on connected endpoint most requested videos

## Local Search (TODO: try tabu list, try removing cache duplicates, try increasing objective swapping)

## Genetic algorithm